# Praca domowa 4
Bartosz Siński

In [ ]:
import dalex 
import pandas as pd
import numpy as np
np.set_seed = 42
from sklearn.model_selection import train_test_split

### Zbiór apratments z DALEX
Na zbiorze danych będziemy przewidywac cenę metra kwadratowego mieszkania, przy użyciu Epsilon-Support Vector Regression.

In [ ]:
df_apartments = dalex.datasets.load_apartments()
df_apartments_test = dalex.datasets.load_apartments_test()

In [ ]:
df_apartments

In [ ]:
df_apartments = pd.get_dummies(df_apartments)
df_apartments_test = pd.get_dummies(df_apartments_test)

In [ ]:
X = df_apartments.drop('m2_price',axis=1)
y = df_apartments[['m2_price']]
X_test = df_apartments_test.drop('m2_price',axis=1)
y_test = df_apartments_test[['m2_price']]
X_train, X_val, y_train, y_val = train_test_split(X, y,random_state = 42)

### SVR z domyślnymi parametrami, standaryzacja i normalizacja danych

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
first_regression = SVR(kernel="poly",degree=5)
first_regression.fit(X_train,y_train)
mean_squared_error(first_regression.predict(X_val),y_val)

Sprawdzimy czy standaryzacja poprawi wynik.

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
scaler2 = preprocessing.StandardScaler().fit(X_test)
X_scaled = scaler.transform(X)
y_scaled = y
X_test_scaled =scaler2.transform(X_test)
y_test_scaled = y_test
X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(X_scaled, y_scaled,random_state = 42)

In [ ]:
second_regression = SVR()
second_regression.fit(X_train_s,y_train_s)
mean_squared_error(first_regression.predict(X_val_s),y_val_s)

A następnie znormalizujmy nasze dane.

In [ ]:
X_n = preprocessing.normalize(X,axis=0)
y_n = y
X_test_n = preprocessing.normalize(X_test,axis=0)
y_test_n = y_test
X_train_n, X_val_n, y_train_n, y_val_n = train_test_split(X_n, y_n,random_state = 42)

In [ ]:
third_regression = SVR()
third_regression.fit(X_train_n,y_train_n)
mean_squared_error(third_regression.predict(X_val_n),y_val_n)

### SVR z tuningiem hiperparametrów

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import RandomizedSearchCV
fourth_regression = SVR()
grid = { 'C' : [0.001,0.01,0.1,10,100,1000],
              'kernel' : ["rbf"],
              'gamma' : [0.0001,0.001,0.01,0.1,1,10,100,'auto','scale']
}
fourth_regression = RandomizedSearchCV(estimator = fourth_regression,param_distributions = grid, cv=5,n_iter=100, random_state=1613,verbose=1)
fourth_regression.fit(X_train,y_train)
mean_squared_error(third_regression.predict(X_val),y_val)

In [ ]:
fourth_regression.best_params_

Do trenowania modelu wykorzystaliśmy jedynie jądro gaussowskie, ponieważ inne jądra powodowały znaczące wydłużenie się czasu trenowania modelu.

### Wyniki 

In [ ]:
print("Base SVR: " + str(mean_squared_error(first_regression.predict(X_test),y_test)))
print("SVR ze standaryzacją zmiennych: " + str(mean_squared_error(second_regression.predict(X_test_scaled),y_test_scaled)))
print("SVR ze normalizacją zmiennych: " + str(mean_squared_error(third_regression.predict(X_test_n),y_test_n)))
print("SVR z tuningiem hiperparametrów: " + str(mean_squared_error(fourth_regression.predict(X_test),y_test)))

W tym przypadku ręczna standaryzacja poprawiła RMSE naszego wyniku lepiej niż normlalizacja. Tuning hiperparametrów także w znaczący sposób poprawił jakość predykcji naszego modelu, pomimo rozważania przez nas jedynie jądra gaussowskiego

### Wybrany przeze mnie zbiór danych - Mobile Price Classification
Zbiór zawiera informacje o telefonach komórkowych, gdzie trzeba zaklasyfikować   telefon do danego zakresu cenowego. 
https://www.kaggle.com/iabhishekofficial/mobile-price-classification

In [ ]:
df_phones = pd.read_csv("./src/phones_train.csv")

In [ ]:
df_phones

In [ ]:
X = df_phones.drop('price_range',axis=1)
y = df_phones[['price_range']]
X_train, X_test,y_train,y_test = train_test_split(X, y,random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,random_state = 42)

### Bazowy SVC, standaryzacja danych i normalizacja danych

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
first_classification = SVC(random_state=42)
first_classification.fit(X_train,y_train)
accuracy_score(first_classifiaction.predict(X_val),y_val)

Następnie klasyfikacja z wcześniejsza standaryzacją danych.

In [ ]:
X_scaled = preprocessing.StandardScaler().fit_transform(X)
X_train_s, X_test_S,y_train_s,y_test_S = train_test_split(X_scaled, y,random_state = 42)
X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(X_train_s, y_train_s,random_state = 42)

In [ ]:
second_classification = SVC(random_state=42)
second_classification.fit(X_train_s,y_train_s)
accuracy_score(second_classification.predict(X_val_s),y_val_s)

I normalizacją.

In [ ]:
X_n = preprocessing.normalize(X,axis=0)
X_train_n, X_test_n,y_train_n,y_test_n = train_test_split(X_n, y,random_state = 42)
X_train_n, X_val_n, y_train_n, y_val_n = train_test_split(X_train_n, y_train_n,random_state = 42)

In [ ]:
third_classification = SVC(random_state=42)
third_classification.fit(X_train_n,y_train_n)
accuracy_score(third_classification.predict(X_val_n),y_val_n)

In [ ]:
pd.DataFrame(X_train_n)

### SVC z tuningiem hiperparametrów

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import RandomizedSearchCV
fourth_classification = SVC(random_state=42)
grid = { 'C' : [0.001,0.01,0.1,10,100,1000],
              'kernel' : ["rbf","poly"],
              'gamma' : [0.0001,0.001,0.01,0.1,1,10,100,'auto','scale'],
            'degree' : [1,2,3,4,5]
}
fourth_classification = RandomizedSearchCV(estimator = fourth_classification,param_distributions = grid, cv=5,n_iter=100, random_state=1613,verbose=1)
fourth_classification.fit(X_train,y_train)
accuracy_score(fourth_classification.predict(X_val),y_val)

In [ ]:
fourth_classification.best_params_

Na zbiorze walidacyjnym widać, że nasz tuning, nie podniósł za bardzo accuracy naszego modelu. Może to być spowodowane bardzo mocnym rozstrzałem w wartościach dobieranych parametrów. Spróbujemy więc jeszcze raz z wartościami bliżej tych, które przed chwilą otrzymalismy.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import RandomizedSearchCV
fifth_classification = SVC(random_state=42)
grid = { 'C' : [0.0001,0.0005,0.001,0.002,0.005,0.7,0.01],
              'kernel' : ["poly"],
              'gamma' : [0.00001,0.00005,0.0001,0.0002,0.0005,0.0007,0.001,'auto','scale'],
            'degree' : [1,2,3]
}
fifth_classification = RandomizedSearchCV(estimator = fifth_classification,param_distributions = grid, cv=5,n_iter=100, random_state=1613,verbose=1)
fifth_classification.fit(X_train,y_train)
accuracy_score(fifth_classification.predict(X_val),y_val)

In [ ]:
fifth_classification.best_params_

### Wyniki

In [ ]:
print("Base SVC: " + str(accuracy_score(first_classification.predict(X_test),y_test)))
print("SVC ze standaryzacją zmiennych: " + str(accuracy_score(second_classification.predict(X_test_S),y_test_S)))
print("SVC ze normalizacją zmiennych: " + str(accuracy_score(third_classification.predict(X_test_n),y_test_n)))
print("SVC z tuningiem hiperparametrów: " + str(accuracy_score(fourth_classification.predict(X_test),y_test)))
print("SVC z tuningiem hiperparametrów++: " + str(accuracy_score(fifth_classification.predict(X_test),y_test)))

Standaryzacja zdecydowanie obniżyła wynik accuracy naszego modelu. Może to być spowodowane istnieniem w naszych danych wielu zmiennych przyjmujących jedynie wartości 0 i 1, które są dalekie od rozkładu normlanego. Normalizacja zmiennych też obniżyła accuracy modelu. Z kolei pierwszy tuning hiperparametrów podniósł accuracy o 1,4%. Kolejny tuning z dokładniejszymi parametrami dodatkowo podniósł accuracy naszego modelu na zbiorze testowym.